In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
pd.set_option("display.max_rows", 101)
pd.set_option("display.max_columns", 101)
%matplotlib inline

In [30]:
import cv2
DIR1 = 'train/one/'
DIR2  = 'train/two/'
DIR3 = 'train/three/'
#画像の読み込み
X=[]
y=[]
for filename in os.listdir(DIR1):
    img = cv2.imread("train/one/{}".format(filename),cv2.IMREAD_GRAYSCALE)
    img=cv2.resize(img, (128, 128), interpolation=cv2.INTER_AREA)
    X.append(img)
    y.append(0)

for filename in os.listdir(DIR2):
    img = cv2.imread("train/two/{}".format(filename),cv2.IMREAD_GRAYSCALE)
    img=cv2.resize(img, (128, 128), interpolation=cv2.INTER_AREA)
    X.append(img)
    y.append(1)
    
for filename in os.listdir(DIR3):
    img = cv2.imread("train/three/{}".format(filename),cv2.IMREAD_GRAYSCALE)
    #img=img.resize((128,128))
    img=cv2.resize(img, (128, 128), interpolation=cv2.INTER_AREA)
    X.append(img)
    y.append(2)
    
X = np.array(X)/255
X=X.astype("float32")
X = np.expand_dims(np.array(X),axis=3)   

In [31]:
X.shape

(3483, 128, 128, 1)

In [35]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import optimizers

In [36]:
batch_size = 32
num_classes = 3
epochs = 100
data_augmentation = True
num_predictions = 20

# The data, split between train and test sets:

# Convert class vectors to binary class matrices.

#y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
#opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
adam=optimizers.Adam()
# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

#x_train = x_train.astype('float32')
#x_test = x_test.astype('float32')
#x_train /= 255
#x_test /= 255


In [85]:
y_train = keras.utils.to_categorical(y, 3)
X_train, X_test, y_train, y_test = train_test_split(X, y_train, stratify=y, test_size=0.2, random_state=942)

In [42]:
model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=10,
              validation_data=(X_test, y_test),
              shuffle=True)

Train on 2786 samples, validate on 697 samples
Epoch 1/10
2786/2786 [==============================] - 82s 30ms/step - loss: 1.0513 - acc: 0.4910 - val_loss: 0.7870 - val_acc: 0.6184
Epoch 2/10
2786/2786 [==============================] - 81s 29ms/step - loss: 0.7050 - acc: 0.6809 - val_loss: 0.5616 - val_acc: 0.7547
Epoch 3/10
2786/2786 [==============================] - 77s 28ms/step - loss: 0.5024 - acc: 0.7879 - val_loss: 0.3269 - val_acc: 0.8852
Epoch 4/10
2786/2786 [==============================] - 79s 28ms/step - loss: 0.3286 - acc: 0.8711 - val_loss: 0.2274 - val_acc: 0.9182
Epoch 5/10
2786/2786 [==============================] - 80s 29ms/step - loss: 0.2113 - acc: 0.9210 - val_loss: 0.1780 - val_acc: 0.9326
Epoch 6/10
2786/2786 [==============================] - 81s 29ms/step - loss: 0.1766 - acc: 0.9368 - val_loss: 0.1622 - val_acc: 0.9484
Epoch 7/10
2786/2786 [==============================] - 79s 28ms/step - loss: 0.1648 - acc: 0.9343 - val_loss: 0.1361 - val_acc: 0.9555
E

In [43]:
model.save("model1.h5")

In [93]:
DIR3 = 'test'
#画像の読み込み
X_test=[]
for filename in os.listdir(DIR3):
    img=cv2.imread("test/{}".format(filename),cv2.IMREAD_GRAYSCALE)
    #img=img.resize((128,128))
    img=cv2.resize(img, (128, 128), interpolation=cv2.INTER_AREA)
    X_test.append(img)
X_test = np.array(X_test)/255
X_test=X_test.astype("float32")
X_test = np.expand_dims(np.array(X_test),axis=3)   

In [46]:
X_test.shape

(871, 128, 128, 1)

In [94]:
preds=model.predict(X_test)
#preds_flip = model.predict(np.array([np.fliplr(x) for x in X_test]))
#tmp=np.array([np.fliplr(x) for x in preds_flip])
#preds=(preds+preds_flip)/2

In [95]:
preds2=np.argmax(preds,axis=1)

In [96]:
d={0:'one',1:"two",2:"three"}

In [97]:
preds3=[d[i] for i in preds2]

In [99]:
submit=pd.DataFrame()
submit["id"]=os.listdir(DIR3)
#submit["id"]=dir3
submit["class"]=preds3

In [100]:
submit.to_csv("fin1v4.csv",index=False)